In [1]:
import cv2
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
from starter_code.visualize import visualize
from starter_code.utils import load_case
from tqdm import tqdm
import tensorflow as tf

In [2]:
volume, segmentation = load_case(123)
X = volume.get_data()
y = segmentation.get_data()

IMG_WIDTH = X.shape[2]
IMG_HEIGHT = X.shape[1]
IMG_SLICES = X.shape[0]
# X = np.expand_dims(X, 3)
# y = tf.keras.utils.to_categorical(y)

In [3]:
from NeuralNetwork import *
model = makeModel(IMG_WIDTH, IMG_HEIGHT, 1)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [4]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, monitor='accuracy'),
    tf.keras.callbacks.TensorBoard(log_dir='logs'),
    tf.keras.callbacks.ModelCheckpoint('saved_models/model1.h5', verbose=1, save_best_only=True)
]

0-179 Train set, 180-209 val_set, 210-299 test

In [5]:
case_numbers = np.delete(np.arange(0, 209, 1), [158, 159, 160, 170, 202])
case_numbers_val = case_numbers[179:]
case_numbers_train = case_numbers[:179]

In [6]:
for n_img in tqdm(case_numbers_train):    
    volume, segmentation = load_case(n_img)
    X = volume.get_data()
    y = segmentation.get_data()
    X = np.expand_dims(X, 3)
    y = tf.keras.utils.to_categorical(y)
    print('\nloaded case {} as train data'.format(n_img))
    random_val_index = case_numbers_val[np.random.randint(0, len(case_numbers_val))]
    volume_val, segmentation_val = load_case(random_val_index)
    X_val = volume_val.get_data()
    y_val = segmentation_val.get_data()
    X_val = np.expand_dims(X_val, 3)
    y_val = tf.keras.utils.to_categorical(y_val)
    print('\nloaded case {} as validation data'.format(random_val_index))
    results = model.fit(X, y, batch_size=16, epochs=4, callbacks=callbacks, verbose=1, validation_data=(X_val, y_val))

0%|          | 0/179 [00:00<?, ?it/s]loaded case 0 as train data
loaded case 1 as validation data
Train on 611 samples, validate on 161 samples
Epoch 1/4
112/611 [====>.........................] - ETA: 49s - loss: 0.5209 - accuracy: 0.8050WARNING:tensorflow:Can save best model only with val_loss available, skipping.


KeyboardInterrupt: 

In [ ]:
model.save('saved_models\model_big_3')

In [ ]:
predicted = model.predict(X)

In [ ]:
def dice_score(y_pred, y_test):
    return np.sum(y_pred)*2.0 / (np.sum(y_pred) + np.sum(y_test))

In [ ]:
dice_score(predicted[:,:,:,1], y[:,:,:,1])

In [ ]:
%matplotlib qt
from visualizeSlider import *
cube_show_slider(cube=predicted[:,:,:,1], axis=0, cmap='gray')

In [ ]:
# volume, segmentation = load_case(123)
# X = volume.get_data()
# X = np.expand_dims(X, 3)
# model = tf.keras.models.load_model('model_big_1')

# print(segmentation.shape)
# nifty = nib.Nifti1Image(segmentation, volume.affine, volume.header)